### Import libraries

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import KBinsDiscretizer
import re
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler


### Read the datasets

In [ ]:
businessdata = pd.read_csv('business.csv')
businessdata['business'] = 1
economy = pd.read_csv('economy.csv')
economy['business'] = 0
dataset = pd.concat([businessdata,economy])

# Data Preparation

#### Rename Columns

In [ ]:
dataset.rename(columns= {'date':'date_travel','ch_code':'character_code', 'num_code':'numerical_code', 'dep_time':'departure_time', 
'from':'source_city', 'time_taken':'duration_flight', 'stop':'stops', 'arr_time':'arrival_time', 'to':'destination_city'},inplace = True)

In [ ]:
dataset.head()

In [ ]:
dataset.count()

#### Dataset Size Reduction and Duplicates Removing

In [ ]:


# duplicate row filtering
print("len before: " + str(len(dataset)))
dataset.drop_duplicates(inplace=True)
print("len after: " + str(len(dataset)))

### Stratified Sampling

In [ ]:
dataset.groupby('price', group_keys=False).apply(lambda x: x.sample(frac=0.9))

#### Type conversion: Change cells from 'stop' column to an ordinal value 


In [ ]:
stopValues = dataset.groupby(['stops'])['stops'].count()
print(stopValues)

**As seen above, some of the cells in the "stop" column have a misformatting error (with multiple '\n' and '\t')**

In [ ]:
dataset['stops'] = dataset['stops'].str.replace('\n', '')
dataset['stops'] = dataset['stops'].str.replace('\t', '')

# dataset.to_csv('stopvalues.csv')

**Now, let's convert to numeric type the number of stops**

In [ ]:
dataset['stops'] = dataset['stops'].str.replace(r'1(.)+',  '1', regex=True)
dataset['stops'] = dataset['stops'].str.replace(r'no(.)+', '0', regex=True)
dataset['stops'] = dataset['stops'].str.replace(r'2(.)+',  '2',regex=True)

dataset['stops'] = dataset['stops'].apply(int)

#### Price to int

In [ ]:

priceValues = dataset.groupby(['price'])['price'].count()
print(priceValues)


dataset['price'] = dataset['price'].str.replace(',', '')
dataset['price'] = dataset['price'].apply(int)




#### Preço do bilhete ao fim de semana e à semana

In [ ]:
dataset['date_travel'] = pd.to_datetime(dataset['date_travel'], format="%d-%m-%Y")

dw_mapping={
    0: 0, 
    1: 0, 
    2: 0, 
    3: 0, 
    4: 0,
    5: 1, 
    6: 1
} 
dataset['weekend']=dataset['date_travel'].dt.weekday.map(dw_mapping)



#### duration_flight to int

In [ ]:
def to_minutes(time_string):
    result = re.search(r'(\d+)h.*?(\d+)m', time_string)
    if (result):
       minutes = int(result.group(1)) * 60 + int(result.group(2))
       return minutes
    else: 
        return -1

dataset['duration_flight'] =  dataset['duration_flight'].apply(to_minutes)
# remover linhas que nao seja possivel converter para int
dataset = dataset[dataset.duration_flight != -1]



### Tratamento à data

In [ ]:
dataset['year'] = dataset['date_travel'].dt.year
dataset['month'] = dataset['date_travel'].dt.month
dataset['day'] = dataset['date_travel'].dt.day

dataset = dataset.drop(['date_travel'], axis=1)

### Tratamento Departure Time and Arrival Time

In [ ]:
def separate_hour(time):
    res = re.search(r'(\d+):(\d+)', time)
    if res:
        return int(res.group(1))

def separate_min(time):
    res = re.search(r'(\d+):(\d+)', time)
    if res:
        return int(res.group(2))



dataset['departure_hour'] = dataset['departure_time'].apply(separate_hour)
dataset['departure_min'] = dataset['departure_time'].apply(separate_min)
dataset = dataset.drop(['departure_time'],axis=1)

dataset['arrival_hour'] = dataset['arrival_time'].apply(separate_hour)
dataset['arrival_min'] = dataset['arrival_time'].apply(separate_min)
dataset = dataset.drop(['arrival_time'],axis=1)


# Dataset Visualization and Analysis

In [ ]:
dataset.to_csv('dataset.csv')

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.head()

#### Analise do preço dos bilhetes


In [ ]:
sns.histplot(data = dataset['price'], bins=10)

#### Relação airline-bilhetes

In [ ]:
sns.histplot(data=dataset, x="airline", hue="business", multiple="dodge")


É possivel verificar que as companhias Vistara e AirIndia têm os preços com a maior variação e os voos mais caros

In [ ]:
plt.figure(figsize = (10,5))
sns.boxplot(data=dataset, x="airline", y="price")
plt.title("Airlines with Prices")
plt.show()

In [ ]:
plt.figure(figsize = (6,5))
sns.boxplot(data=dataset, y="duration_flight")
plt.show()

In [ ]:
plt.figure(figsize = (6,5))
sns.barplot(data=dataset, x="business", y="price")
plt.title("Class with Prices")
plt.show()

#### Analise do preço dos bilhetes

In [ ]:
plt.figure(figsize = (10,5))
estimator = preprocessing.KBinsDiscretizer(n_bins=10, encode="ordinal", strategy='quantile') # testar com onehot encoding
dataset['price_binned'] = estimator.fit_transform(dataset[['price']])
# print(estimator.bin_edges_[0])
dataset.groupby(by=['price_binned']).count()

sns.barplot(x='airline',y='price_binned',hue="business",data=dataset.sort_values("price_binned")).set_title('Airline prices based on the class and company',fontsize=13)

### Preço a semana e ao fim de semana

In [ ]:
sns.barplot(data=dataset, x="weekend", y="price")

#como este atributo não tem grande variação obptamos por fazer o drop

dataset = dataset.drop(['weekend'], axis=1)

In [ ]:

sns.barplot(data=dataset, x="airline", y="price")



### Label Encoding 

In [ ]:
dataset.to_csv('before.csv')

label_encoder = preprocessing.LabelEncoder()
label_encoder.inverse_transform

dataset['airline'] = label_encoder.fit_transform(dataset['airline'])
dataset['character_code'] = label_encoder.fit_transform(dataset['character_code'])
dataset['source_city'] = label_encoder.fit_transform(dataset['source_city'])
dataset['destination_city'] = label_encoder.fit_transform(dataset['destination_city'])
 


#### Searching for missing values

In [ ]:

print(dataset.isnull().sum())

dataset = dataset.dropna()

print(dataset.isnull().sum())

### Correlation Matrix

In [ ]:

corr_matrix = dataset.corr()
f , ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_matrix, vmin=-1, vmax=1, square=True, annot=True)

yearValues1 = dataset.groupby(['year'])['year'].count()
print(yearValues1)

#remove data 
dataset = dataset.drop(['year'], axis=1)
dataset.info()


# Model


In [ ]:
X = dataset.drop(['price','price_binned'], axis=1)
y = dataset['price'].to_frame()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2022)


#### Decision Tree Classifier

In [ ]:

dTree_clf = DecisionTreeRegressor(random_state=2022)
dTree_clf.fit(X_train,y_train)

y_pred = dTree_clf.predict(X_test)


print("R^2 : ", r2_score(y_test, y_pred))
print("MAE :", mean_absolute_error(y_test,y_pred))



### Random Forest Regression

In [ ]:

# Perform Grid-Search
gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='r2', verbose=0, n_jobs=-1)
    
grid_result = gsc.fit(X, y.values.ravel())
best_params = grid_result.best_params_

print(best_params)

rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"], random_state=False, verbose=False)
# Perform K-Fold CV
scores = cross_val_score(rfr, X, y, cv=10, scoring='r2')


#### XGBoost

In [ ]:


model = xgb.XGBRegressor(max_depth = 11, eta = 0.4)

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=2022)
# evaluate model
scores = cross_val_score(model, X, y, scoring='r2', cv=cv)

print("R^2 : ", r2_score(y_test, y_pred))
print("MAE :", mean_absolute_error(y_test,y_pred))




